In [3]:
import pandas as pd
import pyrankvote
from pathlib import Path
from random import seed
seed(252)

In [4]:
df = pd.read_csv(Path('./data/20211114.csv').resolve())
df

,Timestamp,Rank your movie choices. You don't have to do a full ranking. [First Choice],Rank your movie choices. You don't have to do a full ranking. [Second Choice],Rank your movie choices. You don't have to do a full ranking. [Third Choice],What theme should we explore next week? [First Choice],What theme should we explore next week? [Second Choice],What theme should we explore next week? [Third Choice],What theme should we explore next week? [Fourth Choice]
0,09/11/2021 19:45:32,Beau Travail (1999),Shame (2011),NaN,SECRET SOCIETIES,SAD GIRL AUTUMN,DOCUMENTARY,[Default to Trish Pic]
1,09/11/2021 21:27:41,Shame (2011),Beau Travail (1999),Y Tu Mamá También (2001),SAD GIRL AUTUMN,DOCUMENTARY,SECRET SOCIETIES,[Default to Trish Pic]
2,10/11/2021 07:01:47,Y Tu Mamá También (2001),Shame (2011),Beau Travail (1999),SAD GIRL AUTUMN,SECRET SOCIETIES,[Default to Trish Pic],DOCUMENTARY
3,10/11/2021 19:20:14,Y Tu Mamá También (2001),Beau Travail (1999),Shame (2011),SECRET SOCIETIES,DOCUMENTARY,NaN,NaN


In [5]:
just_ballots = df.loc[:, df.columns.str.startswith('Rank')]
next_week = df.loc[:, df.columns.str.startswith('What')]

In [6]:
submissions = just_ballots.loc[just_ballots.notna().all(axis=1).idxmax(),:].values
submissions

array(['Shame (2011)', 'Beau Travail (1999)', 'Y Tu Mamá También (2001)'],
      dtype=object)

In [7]:
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}

In [8]:
balloted = just_ballots.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)

In [9]:
submission_dict

{'Shame (2011)': <Candidate('Shame (2011)')>,
 'Beau Travail (1999)': <Candidate('Beau Travail (1999)')>,
 'Y Tu Mamá También (2001)': <Candidate('Y Tu Mamá También (2001)')>}

In [10]:
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))

In [11]:
ballots

[<Ballot(Beau Travail (1999), Shame (2011))>,
 <Ballot(Shame (2011), Beau Travail (1999), Y Tu Mamá También (2001))>,
 <Ballot(Y Tu Mamá También (2001), Shame (2011), Beau Travail (1999))>,
 <Ballot(Y Tu Mamá También (2001), Beau Travail (1999), Shame (2011))>]

In [12]:
election_result = pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,)

In [13]:
print(election_result)

FINAL RESULT
Candidate                   Votes  Status
------------------------  -------  --------
Y Tu Mamá También (2001)        2  Elected
Beau Travail (1999)             1  Rejected
Shame (2011)                    1  Rejected



In [14]:
submissions = next_week.loc[next_week.notna().all(axis=1).idxmax(),:].values
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}
balloted = next_week.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))
print(pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,))

FINAL RESULT
Candidate                 Votes  Status
----------------------  -------  --------
SECRET SOCIETIES              2  Elected
SAD GIRL AUTUMN               2  Rejected
DOCUMENTARY                   0  Rejected
[Default to Trish Pic]        0  Rejected

